In [1]:
val path = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
// note: use sbt assembly to create jar
interp.load.cp(os.Path(path))
import $ivy.`org.plotly-scala::plotly-almond:0.8.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import ujson.Value
import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._



path: String = "/Users/shawnmeier/Documents/source/bounder/soot_hopper/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                      

import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import ujson.Value

import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")

import $ivy.$                                         

import better.files._



In [52]:
import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.RunConfig
import upickle.default.{macroRW, read, write, ReadWriter => RW}
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._
import edu.colorado.plv.bounder.BounderUtil
import edu.colorado.plv.bounder.symbolicexecutor.state.ReceiverNonNull
import edu.colorado.plv.bounder.symbolicexecutor.state.InitialQuery
import edu.colorado.plv.bounder.TestSpec

import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.RunConfig

import upickle.default.{macroRW, read, write, ReadWriter => RW}

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

import edu.colorado.plv.bounder.BounderUtil

import edu.colorado.plv.bounder.symbolicexecutor.state.ReceiverNonNull

import edu.colorado.plv.bounder.symbolicexecutor.state.InitialQuery

import edu.colorado.plv.bounder.TestSpec

In [67]:
// docker run --rm -it bounder bash // run docker removing on exit
// note: make sure to build the Infer Docker container: https://github.com/facebook/infer/tree/main/docker
val expDir = File(s"${System.getProperty("user.home")}/Desktop/")
def tgt(name:String,apkPath:File,outFolder:File) = name match{
    case x if x matches "AntennapodPlayerFragment_.*" => // _bug or _fix
        val tgt:InitialQuery = ReceiverNonNull(className="com.example.row1antennapodrxjava.ui.main.PlayerFragment", methodName="void call(java.lang.Object)",line=69,receiverMatcher=Some("act"))
        val rc = RunConfig(apkPath=apkPath.toString, outFolder=Some(outFolder.toString), specSet=TestSpec("AntennaPod"), initialQuery= List(tgt))
        write(rc)
    case _ => throw new IllegalArgumentException(s"name: ${name} is not valid")
}

def setupTempDirForSimplified(name:String):File = {
    val workingDir = expDir / (name + "_workingdir")
    if(workingDir.exists())
        workingDir.delete()
    workingDir.createDirectoryIfNotExists()
    val baseSimpAppDir = File((System.getProperty("user.dir").split("/").dropRight(2) ++ Array("test_apps","simplified")).mkString("/"))
    val srcDir = baseSimpAppDir / name
    val wkSRC = workingDir / "src"
    srcDir.copyTo(wkSRC)
    val apkFile = srcDir.glob("**/*.apk").toList
    assert(apkFile.size == 1, "Please build sample app ${name} and ensure build generates apk file.")
    val wkAPK = workingDir / "target.apk"
    apkFile.head.copyTo(wkAPK)
    val wkCFG = workingDir / "cfg.json"
    wkCFG.overwrite(tgt(name, wkAPK, workingDir))
    workingDir
}

def runBounderOnTmp(tmp:File){
    //TODO: call bounder from docker
    Driver.readDB(tmp)
}

def runInferOnTmp(tmp:File){
    ???
}



expDir: File = /Users/shawnmeier/Desktop
defined function tgt
defined function setupTempDirForSimplified
defined function runBounderOnTmp
defined function runInferOnTmp

In [68]:
setupTempDirForSimplified("AntennapodPlayerFragment_fix")

res67: File = /Users/shawnmeier/Desktop/AntennapodPlayerFragment_fix_workingdir

In [64]:
runBounderOnTmp(File("/Users/shawnmeier/Desktop/AntennapodPlayerFragment_fix_workingdir"))

Writing trace 0 / 23, length: 23
Writing trace 1 / 23, length: 23
Writing trace 2 / 23, length: 23
Writing trace 3 / 23, length: 25
Writing trace 4 / 23, length: 27
Writing trace 5 / 23, length: 33
Writing trace 6 / 23, length: 33
Writing trace 7 / 23, length: 39
Writing trace 8 / 23, length: 39
Writing trace 9 / 23, length: 39
Writing trace 10 / 23, length: 39
Writing trace 11 / 23, length: 41
Writing trace 12 / 23, length: 43
Writing trace 13 / 23, length: 45
Writing trace 14 / 23, length: 45
Writing trace 15 / 23, length: 49
Writing trace 16 / 23, length: 49
Writing trace 17 / 23, length: 49
Writing trace 18 / 23, length: 55
Writing trace 19 / 23, length: 65
Writing trace 20 / 23, length: 71
Writing trace 21 / 23, length: 87
Writing trace 22 / 23, length: 87


In [54]:
// dump config files for each
// case class RunConfig(apkPath:String = "",
//                      outFolder:Option[String] = None,
//                      componentFilter:Option[Seq[String]] = None,
//                      specSet: SpecSetOption = TopSpecSet,
//                      initialQuery: List[InitialQuery] = Nil,
//                      limit:Int = -1,
//                      samples:Int = 5,
//                      tag:String = "",
//                      timeLimit:Int = 600, // max clock time per query
//                      truncateOut:Boolean = true
//                     ){


tgt("AntennapodPlayerFragment_bug",File("dummyapk"), File("/home/"))

defined function tgt
res53_1: Any = "{\"apkPath\":\"/Users/shawnmeier/Documents/source/bounder/notebooks/distAppExp/dummyapk\",\"outFolder\":[\"/home\"],\"specSet\":\"testSpec:Antennapod\",\"initialQuery\":[{\"methodName\":\"void call(java.lang.Object)\",\"line\":69,\"t\":\"ReceiverNonNull\",\"matcher\":\"act\",\"className\":\"com.example.row1antennapodrxjava.ui.main.PlayerFragment\"}]}"

In [61]:
def matches(s:String) = s match {
    case x if x matches "foo|b" => true
    case _ => false
}

matches("foo")

defined function matches
res60_1: Boolean = true